[View in Colaboratory](https://colab.research.google.com/github/schwaaweb/aimlds1_08-UnsupervisedLearning/blob/master/Applied_Clustering_CC.ipynb)

## Applied Clustering Coding Challenge

# Clustering text documents using k-means

Clustering is commonly used to create classifications for written text using Natural Language Processing ([NLP](https://learn.lambdaschool.com/ml/sprint/recelerz9zemekmia)) techniques. This approach is highlighted by a combination of supervised and unsupervised techniques - given a huge (and growing) corpus of literature, it may be very hard for observers to classify each document. A clustering approach allows classes to be generated automatically, grouping each document into like clusters. Then, researchers may fit the clustering to specific documents that they wish to get better insigned into. Any document that shares a cluster with the tagged documents of interest then becomes a document of interest.

In this CC, we will do feature extraction and clustering on a collection of newgroup documents.

Text documents are not directly classifiable by ML algorithms, the must first be "Vectorized", that is, turned into numerical vectors by some process. Then each document becomes (as you are now familiar with) a single point in a high dimensional space. Then k-Means or any other ML algorithm can be used for processing. Vectorizing the documents is also called "feature extraction": using either of the two below algorithms a text file is converted into a set of features, aka a numerical row in a database, aka a single sample of `X`, aka a point in a high dimensional space.

Two feature extraction methods can be used in this example:

  - [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) ([Term-Frequency Inverse-Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)) uses a in-memory vocabulary (a python dict) to map the most frequent words to features indices and hence compute a word occurrence frequency (sparse) matrix. The word frequencies are then reweighted using the Inverse Document Frequency (IDF) vector collected feature-wise over the corpus.

  - [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) hashes word occurrences to a fixed dimensional space, possibly with collisions. The word count vectors are then normalized to each have l2-norm equal to one (projected to the euclidean unit-ball) which seems to be important for k-means to work in high dimensional space.

    HashingVectorizer does not provide IDF weighting as this is a stateless
    model (the fit method does nothing). When IDF weighting is needed it can
    be added by pipelining its output to a TfidfTransformer instance.
    
Try both of these feature extractors and compare them:

In [0]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
import sys
from time import time

import numpy as np


# #############################################################################
# Load all categories from the training set
categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

labels = dataset.target

print('Dataset exemplar:')
print(dataset.data[0])

# How many actual classes are there?
true_k = np.unique(labels).shape[0]

## Specify the method of feature extraction and other parameters


In [0]:
use_hashing = True
use_idf = True
n_features = 10000
n_components = 0
minibatchKMeans = True

## Define the feature extractor class and apply it to the  dataset

In [0]:
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()

if use_hashing:
    if use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None, binary=False)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=n_features,
                                       stop_words='english',
                                       alternate_sign=False, norm='l2',
                                       binary=False)
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=n_features,
                                 min_df=2, stop_words='english',
                                 use_idf=use_idf)
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

## Potentially, also do dimensionality reduction

In [0]:
if n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()

## Then do the actual clustering and output clustering metrics

Look at using [MiniBatchKmeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) insead of regular KMeans - MiniBatch means to use a smaller subset of the data over a few passes and average their results. MiniBatch has a shorter runtime than full KMeans and can actually complete on very-large datasets.

Compare the clustering performance metrics between using KMeans and MiniBatchKMeans:
* Homogeneity
* Completeness
* V-measure
* Adjusted Rand-Index
* Silhouette

After you've finished examining the differing performance between KMeans and MiniBatchKMeans, try using a `k` other than `true_k`.

Suggestion: Write a loop that computes and outputs the Clustering Metrics for $k \in \{1...30\}$. Graph one or more of the metrics with $k$ as the `x` axis. We should see them maximize around $k=20$, the true number of clusters. Do we?

Finally, after clustering, draw some exemplar data samples from each cluster and compare them in your own mind. Is the clustering process working for separating out the content of different emails?

In [0]:
# Your implementation of Clustering Execution and Performance Metrics